## 제목을 입력합니다

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from pykrx import stock as st
import numpy as np
import pickle

### csv 불러오기 실습

DataFrame : 2차원 데이터 -> table

In [2]:
stock = pd.read_csv("stock.csv")
stock['num'] = range(0, len(stock))
stock_label = stock[['isin','num']]
stock_label

,isin,num
0,095570,0
1,006840,1
2,027410,2
3,282330,3
4,138930,4
...,...,...
948,005010,948
949,000540,949
950,000547,950
951,000545,951


In [3]:
ohlcv = pd.read_csv('ohlcv최종.csv', dtype={"isin":str})
ohlcv

,isin,s_date,open,high,low,close,volume,amount,updown
0,095570,2015-08-21,30900,36500,28000,35500,1994022,64841739950,14.89
1,095570,2015-08-24,33500,34700,31200,32400,392831,12926256800,-8.73
2,095570,2015-08-25,33050,33650,30950,31100,242730,7808534250,-4.01
3,095570,2015-08-26,31300,38800,31300,35450,703721,25144972800,13.99
4,095570,2015-08-27,36200,40300,34800,37850,499913,18764860300,6.77
...,...,...,...,...,...,...,...,...,...
2212948,003280,2023-06-28,2090,2115,2055,2115,2042994,4239077890,-0.47
2212949,003280,2023-06-29,2135,2135,2055,2055,1388206,2915012720,-2.84
2212950,003280,2023-06-30,2050,2120,2030,2100,1594500,3339019670,2.19
2212951,003280,2023-07-03,2140,2400,2120,2170,21631514,49189201040,3.33


In [4]:
date = pd.DataFrame(ohlcv['s_date'].unique())
date_label = date.rename({0:'s_date'}, axis = 1)
date_label = date_label.sort_values('s_date')
date_label['num'] = range(0, len(date))
date_label = date_label[['s_date','num']]
date_label

,s_date,num
1936,2013-01-02,0
1937,2013-01-03,1
1938,2013-01-04,2
1939,2013-01-07,3
1940,2013-01-08,4
...,...,...
1931,2023-06-28,2581
1932,2023-06-29,2582
1933,2023-06-30,2583
1934,2023-07-03,2584


In [5]:
ohlcv['s_date'] = ohlcv['s_date'].replace(date_label.set_index('s_date')['num'])
ohlcv['isin'] = ohlcv['isin'].replace(stock_label.set_index('isin')['num'])

In [6]:
ohlcv

,isin,s_date,open,high,low,close,volume,amount,updown
0,0,650,30900,36500,28000,35500,1994022,64841739950,14.89
1,0,651,33500,34700,31200,32400,392831,12926256800,-8.73
2,0,652,33050,33650,30950,31100,242730,7808534250,-4.01
3,0,653,31300,38800,31300,35450,703721,25144972800,13.99
4,0,654,36200,40300,34800,37850,499913,18764860300,6.77
...,...,...,...,...,...,...,...,...,...
2212948,952,2581,2090,2115,2055,2115,2042994,4239077890,-0.47
2212949,952,2582,2135,2135,2055,2055,1388206,2915012720,-2.84
2212950,952,2583,2050,2120,2030,2100,1594500,3339019670,2.19
2212951,952,2584,2140,2400,2120,2170,21631514,49189201040,3.33


In [7]:
grouped_df_list = []

for isin, group in ohlcv.groupby('isin'):
    group['y'] = np.where(group['updown'].shift(-1) == 0, 0, 0.5 * (group['updown'].shift(-1) / abs(group['updown'].shift(-1))) + 0.5)
    group = group.dropna(axis=0, how='any')
    group['y'] = group['y'].astype(int)
    grouped_df_list.append(group)

/var/folders/qs/bvkt5pkd1kn1jb4ls1nrvlyc0000gn/T/ipykernel_99186/3997288759.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['y'] = group['y'].astype(int)
/var/folders/qs/bvkt5pkd1kn1jb4ls1nrvlyc0000gn/T/ipykernel_99186/3997288759.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['y'] = group['y'].astype(int)
/var/folders/qs/bvkt5pkd1kn1jb4ls1nrvlyc0000gn/T/ipykernel_99186/3997288759.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [8]:
grouped_df_list[435]

,isin,s_date,open,high,low,close,volume,amount,updown,y
1009708,435,667,114500,116000,106000,113000,41492,4665869000,-1.31,0
1009709,435,668,114500,116500,111000,111500,27335,3090106000,-1.33,0
1009710,435,669,114000,114000,108000,109000,18132,1995071500,-2.24,1
1009711,435,670,110500,111000,108000,109500,9065,988816500,0.46,0
1009712,435,671,107500,109500,107000,107500,8652,935593500,-1.83,0
...,...,...,...,...,...,...,...,...,...,...
1011621,435,2580,96000,96200,95500,96100,475,45504800,0.10,1
1011622,435,2581,96600,96600,95600,96300,379,36357500,0.21,0
1011623,435,2582,96300,96300,94700,95600,1435,136953600,-0.73,1
1011624,435,2583,95200,96600,94600,96200,862,82345100,0.63,0


In [9]:
train = pd.DataFrame([])
test = pd.DataFrame([])
for i, group in enumerate(grouped_df_list):
    if(i % 5 == 0):
        test = pd.concat([test, group])
    else:
        train = pd.concat([train, group])

In [10]:
print(len(train) / (len(test) + len(train)))
print(len(test) / (len(test) + len(train)))

0.8003413200723327
0.19965867992766728


In [11]:
y_train = train.pop("y")
y_train

1936       1
1937       1
1938       1
1939       0
1940       0
          ..
2212947    0
2212948    0
2212949    1
2212950    1
2212951    1
Name: y, Length: 1770355, dtype: int64

In [12]:
x_train = train
x_train

,isin,s_date,open,high,low,close,volume,amount,updown
1936,1,0,21517,21808,21129,21565,15460,341706450,0.23
1937,1,1,21372,22583,21372,22292,37219,844138300,3.37
1938,1,2,22292,23165,22292,22971,46219,1087385800,3.04
1939,1,3,22971,23746,22486,23068,26358,630975900,0.42
1940,1,4,23068,23165,22244,22971,30637,717271900,-0.42
...,...,...,...,...,...,...,...,...,...
2212947,952,2580,1891,2320,1890,2125,41749319,89510302025,12.73
2212948,952,2581,2090,2115,2055,2115,2042994,4239077890,-0.47
2212949,952,2582,2135,2135,2055,2055,1388206,2915012720,-2.84
2212950,952,2583,2050,2120,2030,2100,1594500,3339019670,2.19


In [13]:
y_test = test.pop("y")
y_test

0          0
1          0
2          1
3          1
4          0
          ..
2207775    1
2207776    1
2207777    0
2207778    0
2207779    0
Name: y, Length: 441645, dtype: int64

In [14]:
x_test = test
x_test

,isin,s_date,open,high,low,close,volume,amount,updown
0,0,650,30900,36500,28000,35500,1994022,64841739950,14.89
1,0,651,33500,34700,31200,32400,392831,12926256800,-8.73
2,0,652,33050,33650,30950,31100,242730,7808534250,-4.01
3,0,653,31300,38800,31300,35450,703721,25144972800,13.99
4,0,654,36200,40300,34800,37850,499913,18764860300,6.77
...,...,...,...,...,...,...,...,...,...
2207775,950,2580,9090,9090,8090,8800,22177,189178150,-2.22
2207776,950,2581,8610,9450,8500,8920,20045,184586820,1.36
2207777,950,2582,9900,10040,9200,9480,50108,483141270,6.28
2207778,950,2583,8590,8680,8050,8050,30299,255843070,-15.08


In [15]:
model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

In [16]:
result = round(pd.DataFrame(model.predict(x_test)))
result[0] = result[0].astype(int)
y_test = pd.DataFrame(y_test).reset_index()
y_test = y_test.drop('index', axis=1)
result = pd.concat([result, y_test], axis=1)
result['일치'] = (result[0] == result['y']) * 1
result

,0,y,일치
0,0,0,1
1,0,0,1
2,0,1,0
3,0,1,0
4,0,0,1
...,...,...,...
441640,0,1,0
441641,0,1,0
441642,0,0,1
441643,0,0,1


In [17]:
len(result[result['일치'] == 1])/len(result)

0.5556838637367116

In [18]:
y_test

,y
0,0
1,0
2,1
3,1
4,0
...,...
441640,1
441641,1
441642,0
441643,0


In [19]:
pickle.dump(model, open('predict_updown_model.sav', 'wb'))

In [20]:
model = pickle.load(open('predict_updown_model.sav', 'rb'))

In [21]:
isin = stock_label['isin']
isin

0      095570
1      006840
2      027410
3      282330
4      138930
        ...  
948    005010
949    000540
950    000547
951    000545
952    003280
Name: isin, Length: 953, dtype: object

In [22]:
date_label[date_label['s_date']==max(date_label['s_date'])]

,s_date,num
1935,2023-07-04,2585


In [23]:
df = st.get_market_ohlcv("20230704", "20230707", "005930")
df = df.reset_index()
tmp_map = max(date_label['num'])
df['날짜'] = df['날짜'].astype(str)
for s_date in df['날짜']:
    if s_date not in date_label['s_date'].values:
        new_row = pd.DataFrame({'s_date': [s_date], 'num': [tmp_map + 1]})
        date_label = pd.concat([date_label, new_row], ignore_index=True)
        tmp_map += 1
df['날짜'] = df['날짜'].replace(date_label.set_index('s_date')['num'])
df

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률
0,2585,73400,73600,72900,73000,10214350,747624440200,0.00
1,2586,73000,73300,71900,72000,12310610,889637363400,-1.37
2,2587,71900,72400,71500,71600,14777667,1061491980700,-0.56
3,2588,71100,71400,69800,69900,17308877,1215404338500,-2.37


In [24]:
date_label

,s_date,num
0,2013-01-02,0
1,2013-01-03,1
2,2013-01-04,2
3,2013-01-07,3
4,2013-01-08,4
...,...,...
2584,2023-07-03,2584
2585,2023-07-04,2585
2586,2023-07-05,2586
2587,2023-07-06,2587


In [25]:
new_data = pd.DataFrame([])
date_set = ["20230704","20230705","20230706","20230707"]

for date in date_set:
    df = st.get_market_ohlcv(date)
    df = df.reset_index()
    df["s_date"] = date[:4] + "-" + date[4:6] + "-" + date[6:]
    df = df.dropna(axis=0, how='any')
    new_data = pd.concat([new_data,df])
new_data = new_data.rename({"티커":"isin", "시가" : "open", "고가" : "high", "저가" : "low", "종가" : "close", "거래량" : "volume", "거래대금" : "amount", "등락률" : "updown"}, axis=1)
new_data = new_data[['isin', 's_date', 'open', 'high', 'low', 'close', 'volume', 'amount', 'updown']]
new_data = new_data.sort_values(['isin', 's_date'])
new_data['y'] = np.where(new_data['updown'].shift(-1) == 0, 0, 0.5 * (new_data['updown'].shift(-1) / abs(new_data['updown'].shift(-1))) + 0.5)
new_data = new_data[new_data["s_date"] != "2023-07-07"]
new_data = new_data.reset_index()
new_data.drop(['index'], axis=1, inplace=True)
new_data['isin'] = new_data['isin'].replace(stock_label.set_index('isin')['num'])
new_data['s_date'] = new_data['s_date'].replace(date_label.set_index('s_date')['num'])
new_data['y'] = new_data['y'].astype(int)
new_data

,isin,s_date,open,high,low,close,volume,amount,updown,y
0,340,2585,10300,10440,10260,10390,83550,865107990,0.48,0
1,340,2586,10390,10440,10090,10370,154517,1582256610,-0.19,0
2,340,2587,10390,10390,10190,10300,92202,946743290,-0.68,0
3,84,2585,602,602,577,584,147115,86238821,-1.85,0
4,84,2586,598,600,573,575,227654,132339906,-1.54,0
...,...,...,...,...,...,...,...,...,...,...
2854,590,2586,2430,2460,2340,2440,1623413,3891185490,0.41,0
2855,590,2587,2440,2500,2350,2360,1047983,2491679000,-3.28,1
2856,796,2585,9300,9530,9240,9320,29216,272040120,-0.64,1
2857,796,2586,9230,9710,9200,9390,62709,589109300,0.75,0


In [26]:
y_new_data = new_data.pop("y")
x_new_data = new_data
y_new_data

0       0
1       0
2       0
3       0
4       0
       ..
2854    0
2855    1
2856    1
2857    0
2858    0
Name: y, Length: 2859, dtype: int64

In [27]:
result = pd.concat([pd.DataFrame(model.predict(x_new_data)), y_new_data],axis=1)
result['일치'] = (round(result[0]) == result['y']) * 1

In [28]:
result

,0,y,일치
0,0.443508,0,1
1,0.444988,0,1
2,0.446587,0,1
3,0.451436,0,1
4,0.450540,0,1
...,...,...,...
2854,0.442223,0,1
2855,0.453263,1,0
2856,0.444779,1,0
2857,0.441189,0,1


In [29]:
len(result[result['일치'] == 1])/len(result)

0.7768450507170339

In [30]:
str(round(len(result[result['일치'] == 1])/len(result) * 100,2)) + "%"

'77.68%'